# Perform a differential gene expression analysis with siibra

The DifferentialGeneExpression class compute differential gene expresssion in two different brain regions, following the JuGEx algorithm described in the following publication:

> *Sebastian Bludau, Thomas W. Mühleisen, Simon B. Eickhoff, Michael J. Hawrylycz, Sven Cichon, Katrin Amunts. Integration of transcriptomic and cytoarchitectonic data implicates a role for MAOA and TAC1 in the limbic-cortical network. 2018, Brain Structure and Function. https://doi.org/10.1007/s00429-018-1620-6*
      
In siibra, the object is initialized with a siibra atlas object. It will check if the parcellation selected in the atlas is suitable for performing the analysis, which includes to verify that the given atlas object provides maps in the MNI ICBM 152 space. The analysis is configured by specifying some candidate genes of interest, and two regions of interest (ROI) specified by brain area names that the atlas object can resolve. Note that tha siibra atlas class does fuzzy string matching to resolve region names, so you can try with a simple name of the regions to see if siibra interprets them.  Also, gene names can easily be looked up and autocompleted in siibra.features.gene_names. 

For the gene expression data, siibra accesses the Allen Brain Atlas API (© 2015 Allen Institute for Brain Science. Allen Brain Atlas API. Available from: brain-map.org/api/index.html). 

## Preparation

We install the siibra==0.1a8, siibra-jugex==0.1a2 libraries.

In [ ]:
# This notebook requires you to have the following packages installed
# https://pypi.org/project/siibra/
# https://pypi.org/project/siibra-jugex/

!pip install -U pip
!pip install siibra==0.1a8 siibra-jugex==0.1a2

# Set cache dir. If unset, will use default user tempdir as returned by the appdirs module
# environ['SIIBRA_CACHEDIR'] = "<<<Cache dir>>>"

## Instantiate human brain atlas with Julich-Brain cytoarchitectonic maps

We instantiate the human atlas, and tell it to prefer thresholded continous maps over labelled regions in the discrete parcellation map. For the Julich-Brain parcellation, this mean that it will use thresholded probability maps, if available.

In [ ]:
import siibra
siibra.logger.setLevel("INFO") # we want to see some messages!

In [ ]:
atlas = siibra.atlases['human']

# next line is optional - cytoarchitectonic maps are selected by default
atlas.select_parcellation('julich 2.9')

# as in the original JuGEx, we prefer thresholded probability maps 
# over the labelled region in the maximum probability map

threshold = $THRESHOLD$
atlas.threshold_continuous_maps(threshold)

## Setup and run the experiment

In [ ]:
candidate_regions = [$AREA1$, $AREA2$]
candidate_genes = $GENELIST$
n_rep = $REPS$

Configuring the experiment is now just a matter of a few lines of code:

In [ ]:
import siibra_jugex
jugex = siibra_jugex.DifferentialGeneExpression(atlas)
for gene in candidate_genes:
  jugex.add_candidate_genes(gene)
jugex.define_roi1(candidate_regions[0])
jugex.define_roi2(candidate_regions[1])

We are now setup to run the analysis.

In [ ]:
result = jugex.run(permutations=n_rep)

In [ ]:
print(result['p-values'])

## Look at the anatomical positions of the samples in MNI space

Let's have a look at the sample positions that have been found in the Allen atlas. Since we configured siibra to prefer thresholded continuous maps for region filtering over the simplified parcellation map, we also plot the probability maps here.

In [ ]:
from nilearn import plotting
import numpy as np

for region in candidate_regions:
    samples = jugex.get_samples(region)
    atlas.select_region(region)
    pmap = atlas.selected_region.get_regional_map(
        siibra.spaces['mni152'], siibra.MapType.CONTINUOUS)    
    # we could have also used the simple parcellation map mask as follows:
    # mask = atlas.get_mask(siibra.spaces['mni152'])
    display = plotting.plot_roi(pmap.fetch(),cmap='jet')
    display.add_markers([s['mnicoord'] for s in samples.values()])

The aggregated input parameters can be stored to disk.

In [ ]:
jugex.save('jugex_{}_{}.json'.format(
    "_".join(candidate_regions),
    "_".join(candidate_genes) ))